# This notebook shows how to use Orchestrator APIs for user experiments

## Configure the Environment

In [ ]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token_beta.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='pruth_0031379841'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/FABRIC/pruth-bastion-1'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import FABRIC API

In [ ]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

## Create Slice

## Configure Slice Parameters



In [ ]:
slice_name = 'MySliceL2Net'
site1 = 'STAR'
site2 = 'DALL'



node1_name = 'Node1'
node2_name = 'Node2'



network_name='net1'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'

node1_dataplane_ip = '192.168.1.10'
node2_dataplane_ip = '192.168.1.12'
network_cidr = '24'

#image = 'default_ubuntu_20'
image = 'default_rocky_8'
cores = 8
ram = 32
disk = 100

In [ ]:
 try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
    available_resources.draw()
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

NIC component models options:
- NIC_ Basic 
- NIC_ConnectX_5 
- NIC_ConnectX_6

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=node1_name, site=site1)
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    iface1 = node1.add_component(model='NIC_ConnectX_6', name=node1_nic_name).get_interfaces()[0]
    iface1.set_vlan(vlan='1000')
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site2)
    node2.set_capacities(cores=cores, ram=ram, disk=disk)
    node2.set_image(image)
    iface2 = node2.add_component(model='NIC_ConnectX_6', name=node2_nic_name).get_interfaces()[0]
    iface2.set_vlan(vlan='1000')
    
    # Network
    net1 = slice.add_l2network(name=network_name, interfaces=[iface1, iface2])

    #Submit Slice Request
    slice.submit(wait_progress=False)
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()

In [ ]:
import time

            
try:
    print(f"Waiting for slice {slice_name} ")

    #Get Slice
    slice = fablib.get_slice(slice_name)

    #Wait for ssh to be active
    slice.wait_ssh(progress=True, timeout=600,interval=60)
    #time.sleep(30)
    
    #Run post boo config
    slice.post_boot_config(verbose=True)
except Exception as e:
    print(f"Slice Failed: {e}")
    traceback.print_exc()

## Get the Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print(f"   Components        :  ")
        for component in node.get_components():
            print(f"      Name             : {component.get_name()}")
            print(f"      Details          : {component.get_details()}")
            print(f"      Disk (G)         : {component.get_disk()}")
            print(f"      Units            : {component.get_unit()}")
            print(f"      PCI Address      : {component.get_pci_addr()}")
            print(f"      Model            : {component.get_model()}")
            print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ")
        for interface in node.get_interfaces():
            print(f"       Name                : {interface.get_name()}")
            print(f"           Bandwidth           : {interface.get_bandwidth()}")
            print(f"           VLAN                : {interface.get_vlan()}")      
            print(f"           OS Interface        : {interface.get_os_interface()}")       
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Get the Nodes

Retrieve the node information and save the management IP address.


### Configure Node1

Use ssh to configure eth1 on  node 1.  

```
ip link add link eth1 name eth1.200 type vlan id 200
ip link set dev eth1.200 up
ip addr add 192.168.1.10/24 dev eth1.200

```

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network_name)  
    node1_iface.set_ip(ip=node1_dataplane_ip, cidr=network_cidr, mtu="9000")
    
    stdout, stderr = node1.execute(f'ip addr list')
    print (stdout)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

### Configure Node2

Use ssh to configure eth1 on each Node 2.  

```
ip link add link eth1 name eth1.200 type vlan id 200
ip link set dev eth1.200 up
ip addr add 192.168.1.11/24 dev eth1.200
```

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network_name)  
    node2_iface.set_ip(ip=node2_dataplane_ip, cidr=network_cidr, mtu='9000')
    
    stdout, stderr = node2.execute(f'ip addr list')
    print (stdout)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

### Test Network

#### Util functions

In [ ]:
def mtu_test(node1, node2, node2_ip, verbose=True):
    if verbose: print("Testing MTU:")

    #Run test
    output = {}

    ping_packets_count = 3
    #ping_packet_sizes = [9000, 8950, 8000, 1500, 1450, 1400, 1000, 500, 100, 50]
    max_success = -1
    min_fail = 10000

    #Test min ping
    current_size = 0

    stdout, stderr = node1.execute('ping -M do -s ' + str(current_size) + ' -c ' + str(ping_packets_count) + ' ' + node2_ip + " | grep transmitted")
    ping_string = stdout

    data_array = ping_string.split(" ")
    recieved_packets = 0
    if len(data_array) > 3:
        recieved_packets = data_array[3]

    if(int(recieved_packets) == ping_packets_count):
        max_success = current_size
        min_fail = 10000
    else:
        min_fail = 0
        max_success = -1


    while max_success < min_fail - 1:

        current_size = int((min_fail+max_success)/2)
        #print("min_fail: {}, max_success: {}, current_size: {}".format(str(min_fail),str(max_success), str(current_size)))

        stdout, stderr = node1.execute('ping -M do -s ' + str(current_size) + ' -c ' + str(ping_packets_count) + ' ' + node2_ip + " | grep transmitted")
        ping_string = stdout

        #print("current_size: {}, output: {}".format(str(current_size),ping_string))

        data_array = ping_string.split(" ")
        recieved_packets = 0
        if len(data_array) > 3:
            recieved_packets = data_array[3]

        #print("recieved_packets: {}, ping_packets_count: {}".format(str(recieved_packets),str(ping_packets_count)))
        if(int(recieved_packets) == int(ping_packets_count)):
            max_success = current_size
        else:
            min_fail = current_size

    if max_success > 0:
        output['mtu'] = str(max_success+28)
    else:
        output['mtu'] = 0

    if verbose: print(", mtu: {}".format(output['mtu']),end='')

    return output


def bandwidth_test(node1, node2, node1_ip, node2_ip, verbose=True):
    if verbose: print("Testing Bandwidth:")
    output = {}

    #stdout, stderr = node1.execute('echo "net.core.rmem_max = 2147483647\nnet.core.wmem_max = 2147483647\nnet.ipv4.tcp_rmem = 4096 87380 2147483647\nnet.ipv4.tcp_wmem = 4096 65536 2147483647\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
    #stdout, stderr = node2.execute('echo "net.core.rmem_max = 2147483647\nnet.core.wmem_max = 2147483647\nnet.ipv4.tcp_rmem = 4096 87380 2147483647\nnet.ipv4.tcp_wmem = 4096 65536 2147483647\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')


    stdout, stderr = node1.execute('iperf3 -s > /dev/null 2>&1 &')

    #stdout, stderr = node2.execute('iperf3 -J -c ' + node1_ip + ' -t 60 -P 1 -w 512M')
    stdout, stderr = node2.execute('iperf3 -J -c ' + node1_ip + ' -t 60 -P 1')
    #print(f"stdout {stdout}")
    try:
        results = json.loads(stdout)
        output['forward'] = results
    except Exception as e:
        print("error {}".format(e))
        print("iperf raw stdout: {}".format(stdout))
        traceback.print_exc()

    #stdout, stderr = node2.execute('iperf3 -J -R -c ' + node1_ip + ' -t 60 -P 1 -w 512M')
    stdout, stderr = node2.execute('iperf3 -J -R -c ' + node1_ip + ' -t 60 -P 1')
    try:
        results = json.loads(stdout)
        output['reverse'] = results
    except Exception as e:
        print("error {}".format(e))
        print("iperf raw stdout: {}".format(stdout))
        traceback.print_exc()

    #return {'bandwidth_test': output}
    return output

### Config Networking Tools

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address

try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        if type(ip_address(node.get_management_ip())) is IPv6Address:
            print(f"{node.get_name()}: IPv6")
            #stdout, stderr = node.execute('sudo echo "nameserver 2001:4860:4860::8888" > /etc/resolv.conf && sudo echo "nameserver 2001:4860:4860::8844" >> /etc/resolv.conf')
            #print(f"stdout: {stdout}")
            #print(f"stderr: {stderr}")
            
        #https://srcc.stanford.edu/100g-network-adapter-tuning
        #https://fasterdata.es.net/host-tuning/linux/
        stdout, stderr = node.execute('sudo echo "net.core.rmem_max = 268435456" >  /etc/sysctl.conf')
        stdout, stderr = node.execute('sudo echo "net.core.wmem_max = 268435456" >>  /etc/sysctl.conf')
        stdout, stderr = node.execute('sudo echo "net.ipv4.tcp_rmem = 4096 87380 134217728" >>  /etc/sysctl.conf')
        stdout, stderr = node.execute('sudo echo "net.ipv4.tcp_wmem = 4096 65536 134217728" >>  /etc/sysctl.conf')
        stdout, stderr = node.execute('sudo echo "net.ipv4.tcp_congestion_control=bbr" >>  /etc/sysctl.conf')
        stdout, stderr = node.execute('sudo echo "net.ipv4.tcp_mtu_probing=1" >>  /etc/sysctl.conf')
        stdout, stderr = node.execute('sudo echo "net.core.default_qdisc = fq" >>  /etc/sysctl.conf')
        stdout, stderr = node.execute('sudo echo "net.core.netdev_max_backlog = 250000" >>  /etc/sysctl.conf')
        stdout, stderr = node.execute('net.ipv4.tcp_no_metrics_save = 1" >>  /etc/sysctl.conf')
        stdout, stderr = node.execute('sudo sysctl -p')
        
        print(f"{node.get_interface(network_name=network_name).get_physical_os_interface()['ifname']}")
        print(f"{node.get_interface(network_name=network_name).get_os_interface()}")
        stdout, stderr = node.execute(f"sudo ethtool -K {node.get_interface(network_name=network_name).get_physical_os_interface()['ifname']} lro on")
        stdout, stderr = node.execute(f"sudo ifconfig {node.get_interface(network_name=network_name).get_physical_os_interface()['ifname']} txqueuelen 20000")
        stdout, stderr = node.execute(f"sudo ethtool -K {node.get_interface(network_name=network_name).get_os_interface()} lro on")
        stdout, stderr = node.execute(f"sudo ifconfig {node.get_interface(network_name=network_name).get_os_interface()} txqueuelen 20000")
        stdout, stderr = node.execute(f"sudo systemctl stop irqbalance")
    
        stdout, stderr = node.execute('sudo yum install -y iperf3 net-tools vim')
except Exception as e:
    print(f"{e}")

### Latency Test

In [ ]:
#Latency Test
try:
    node1 = slice.get_node(name=node1_name)    
    node2 = slice.get_node(name=node2_name)  
    node2_iface = node2.get_interface(network_name=network_name)  
  
    stdout, stderr = node1.execute(f'ping -c 3 {node2_dataplane_ip}')
    print (stdout)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

In [ ]:
try:
    print(f"MTU: {mtu_test(node1, node2, node2_dataplane_ip, verbose=False)}")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [ ]:
try:
    iperf_output = bandwidth_test(node1, node2, node1_dataplane_ip, node2_dataplane_ip)
    
    print(f"Forward bandwidth: {float(iperf_output['forward']['end']['sum_received']['bits_per_second'])/1000000000:.3f} Gbps")
    print(f"Reverse bandwidth: {float(iperf_output['reverse']['end']['sum_received']['bits_per_second'])/1000000000:.3f} Gbps")

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

## Delete Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()